#Model

In [30]:
import torch
from torch import nn
import torch.nn.functional as F

In [31]:
#process : images(input_dim) -> hidden dim -> mean and std deviation vectors -> reparametrization -> latent dimension -> output_dim
class VariationalAutoEncoder(nn.Module):
  def __init__(self,input_dim, h_dim = 200,z_dim = 20): #h_dim:hidden_dimension
    super().__init__()

    #encoder
    self.img_2hid = nn.Linear(input_dim,h_dim) #taking input image into a hidden dimension
    self.hid_2mu = nn.Linear(h_dim,z_dim) #z_dim is dimension of mu vector or std deviation vector or the latent vector (all 3 have the same dim)
    self.hid_2sigma = nn.Linear(h_dim,z_dim) #for sd vector

    #reparamterization will be taken care of later(in the forward class).for now,assuming reparamaterization is done,we start from the latent vector in the decoder

    #decoder
    self.z_2hid = nn.Linear(z_dim,h_dim)
    self.hid_2img = nn.Linear(h_dim,input_dim)



  def encode(self,x): #q_phi(z|x)
    h = F.relu(self.img_2hid(x)) #mapping input into hidden dim followed by relu activation
    mu,sigma = self.hid_2mu(h),self.hid_2sigma(h) #?) why not ReLu here
    return mu,sigma
  
  def decode(self,z): #p_theta(x|z) again assuming we already have the latent vector z
    h = F.relu(self.z_2hid(z))
    img = self.hid_2img(h)
    img = torch.sigmoid(img) #this step is specifically for our use-case,that is MNIST dataset,which assumes the pixel values are in b/w 0 and 1
    return img

  def forward(self,x,z):
    #encode
    mu,sigma = self.encode(x)
    #now we apply reparamterization
    epsilon = torch.rand_like(sigma)
    z_reparametrized = mu+sigma*epsilon
    #decode
    x_reconstructed = self.decode(z_reparametrized)
    return x_reconstructed,mu,sigma #remember that we need this mu and sigma to compute the loss functions (especially while computing the KL divergence)

In [32]:
if __name__ == "__main__":
  x = torch.randn(4,28*28) #batch_size = 4 and 28*28 images
  vae = VariationalAutoEncoder(input_dim = 784) #input dim is 28*28 flattened as an input vector. The other two paramters are already initialised above

#Training

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm #for progress bar
import torchvision.datasets as datasets  #to get MNIST
from torch.utils.data import DataLoader

In [34]:
#initializing hyperparameters
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_DIM = 784
H_DIM = 200
Z_DIM = 20
NUM_EPOCHS = 10
BATCH_SIZE = 32
LR_RATE = 3e-4

In [35]:
#Loading dataset
dataset = datasets.MNIST(root = "dataset/",train = True,transform = transforms.ToTensor(),download = True) #.ToTensor also divides by 255(scaling)
train_loader = DataLoader(dataset = dataset,batch_size = BATCH_SIZE,shuffle = True)
model = VariationalAutoEncoder(INPUT_DIM,H_DIM,Z_DIM).to(DEVICE)
optimizer = optim.Adam(model.parameters(),lr = LR_RATE)
loss_fn = nn.BCELoss(reduction = 'sum') #might use MSE also

In [36]:
#Training
for epoch in range(NUM_EPOCHS):
  loop = tqdm(enumerate(train_loader))
  for i,(x,_) in loop:
    #Forward pass
    x = x.to(DEVICE).view(x.shape[0],INPUT_DIM)
    x_reconstructed,mu,sigma = model(x,Z_DIM)

    #Compute loss
    reconstruction_loss = loss_fn(x_reconstructed, x)
    kl_div = -torch.sum(1+torch.log(sigma.pow(2)) - mu.pow(2) - sigma.pow(2)) #-ve sign was not part of the paper but pytorch minimizes loss by default

    #Backpropagation
    loss = reconstruction_loss + kl_div
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loop.set_postfix(loss = loss.item())


1875it [00:32, 57.81it/s, loss=4.3e+3]
1875it [00:31, 59.48it/s, loss=3.59e+3]
1875it [00:32, 57.04it/s, loss=3.73e+3]
1875it [00:32, 58.13it/s, loss=3.24e+3]
1875it [00:32, 58.02it/s, loss=3.51e+3]
1875it [00:32, 57.67it/s, loss=3.49e+3]
1875it [00:31, 58.89it/s, loss=3.43e+3]
1875it [00:32, 57.44it/s, loss=3.16e+3]
1875it [00:31, 58.97it/s, loss=3.33e+3]
1875it [00:32, 57.08it/s, loss=3.47e+3]


In [38]:
def inference(digit, num_examples=1): #digits and number of examples we want to generate

#Generates (num_examples) of a particular digit.Specifically we extract an example of each digit,then after we have the mu, sigma representation foreach digit we can sample from that.

#After we sample we can run the decoder part of the VAE and generate examples.
    images = []
    idx = 0 #getting 10 images
    for x, y in dataset:
        if y == idx:
            images.append(x)
            idx += 1
        if idx == 10:
            break

    encodings_digit = [] #getting mu and sigma for each image
    for d in range(10):
        with torch.no_grad():
            mu, sigma = model.encode(images[d].view(1, 784))
        encodings_digit.append((mu, sigma))

    mu, sigma = encodings_digit[digit] #generate new images after reparametrization
    for example in range(num_examples):
        epsilon = torch.randn_like(sigma)
        z = mu + sigma * epsilon
        out = model.decode(z)
        out = out.view(-1, 1, 28, 28)
        save_image(out, f"generated_{digit}_ex{example}.png")

for idx in range(10):
  inference(idx,num_examples = 1)

In [40]:
from PIL import Image
 
for i in range(10):
  img = Image.open('/content/generated_'+str(i)+'_ex0.png')
  img.show()

 The recreated MNIST images are still blurry due to the slow converge of the overall loss.This shows that the hyperparameters can further be fine-tuned like learning rate,batch_size,number of epochs,etc.. to make the images look more sharp